Todo

- Exportovat seznam seznamových sloupců, aby ho další sešit v pipelině načítal automaticky.

- Ještě mnoho sloupců seznamových, kvůli čárkám ", sro" nutno upravit při scrapování.

- Ukazují se exulantské filmy! Zároveň tam ale chceme nechat koprodukce. (Ten samý problém u herectva.)

- Premiéra do datetime.

- Rok výroby do dvou sloupců.

In [24]:
import pandas as pd

In [37]:
filmy = pd.read_json("filmy_raw.json")

In [38]:
filmy

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Podnázev anglický,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Překlad české verze,Rychlost,Přístupnost,cyklus
0,Tábor padlých žen (10144),"Slovensko, ...",1997,1997,premiéra 28. 9. 2000 /doporučená přístupnost o...,100 min,Laco Halama,film,"historický, tragikomedie","hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
1,Eastern – Krvavý Hugo (10173),Česká republika,1997,1997,nedistribuční premiéra 27. 4. 1997 ...,6 min,Aurel Klimt (1189),film,groteska,"animovaný, distribuční, studentský",...,None,None,None,None,None,None,None,None,None,None
2,O Ječmínkovi (102282),Česká republika,2003,2002,nedistribuční premiéra 18. 4. 2003 (televizní...,61 min,Milan Cieslar (3593),film,pohádka,"hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
3,Zimní víla (102314),Česká republika,1999,1999,nedistribuční premiéra 24. 12. 1999 (televizn...,52 min,Kryštof Hanzlík (1055),film,pohádka,"hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
4,Narkoblues (10242),"Česká republika, ...",1997,1997,premiéra 10. 11. 2016 /přístupné bez omezení/,9 min,"Břetislav Pojar (3113), Ivan Vít (3030)",film,osvětový,"animovaný, distribuční",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,Česká republika,1997,1997,premiéra 12. 8. 1997,None,Rodrigo Morales (3466),film,povídka,"hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
4532,Zdivočelá země (9738),Česká republika,1997,1997,premiéra 13. 11. 1997 /přístupný/ (kino 64 U H...,114 min,Hynek Bočan (2947),film,"dobrodružný, historický","hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
4533,Biograf (9769),Česká republika,1997,1997,nedistribuční premiéra 12. 6. 1997 ...,5 min,Galina Hůlová (7957),film,anekdota,"animovaný, distribuční, studentský",...,None,None,None,None,None,None,None,None,None,None
4534,Knoflíkáři (9793),Česká republika,1997,1997,premiéra 20. 11. 1997 /doporučená přístupnost ...,102 min,Petr Zelenka (4806),film,"komedie, povídkový","hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None


In [39]:
def citlivy_orez(x):
    if isinstance(x, str):
        x = x.strip()
    if isinstance(x, list):
        try:
            x = [y.strip() for y in x]
        except:
            pass
    return(x)

In [40]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [41]:
filmy = filmy.apply(lambda x: citlivy_orez(x))

In [42]:
seznamove_sloupce = ["Žánr",
                     "Režie",
                     "Scénář",
                     "Mluveno",
                     "Barva",
                     "Zvukový systém/formát",
                     "Země původu",
                     "Úvodní/závěrečné titulky",
                     "Distribuční nosič",
                     "Země původu",
                     "Typologie"]

In [43]:
for s in seznamove_sloupce:
    filmy[s] = filmy[s].apply(lambda x: do_seznamu(x))

In [44]:
filmy["Minutáž"] = filmy["Minutáž"].apply(lambda x: str(x).replace(" min",""))
filmy["Minutáž"] = pd.to_numeric(filmy["Minutáž"], errors="coerce")

In [45]:
filmy = filmy.sort_values(by = "Copyright")

In [46]:
filmy.to_json("filmy.json", orient="records")